## 2nd strategy for E.coli 5-HTP production

In [22]:
from cobra.io import read_sbml_model
from cobra.io import write_sbml_model
from cobra import Reaction, Metabolite
import pandas as pd

In [2]:
model = read_sbml_model('../data/iML1515.xml')

**BH4 synthesis pathway**

GTP cyclohydrolase I\
GTP + water <-> dihydromonapterin-triphosphate + formate + H+

In [3]:
#design reaction
gchi = Reaction('gchi')
gchi.add_metabolites({model.metabolites.gtp_c: -1,
                      model.metabolites.h2o_c: -1,
                      model.metabolites.dhmptp_c: 1,
                      model.metabolites.for_c: 1,
                      model.metabolites.h_c: 1
                     })
print(gchi.build_reaction_string())

gtp_c + h2o_c --> dhmptp_c + for_c + h_c


In [4]:
#add reaction to the system
model.add_reactions([gchi])
model.reactions.gchi

Reaction identifier,gchi
Name,
Memory address,0x07f2eeff22880
Stoichiometry,gtp_c + h2o_c --> dhmptp_c + for_c + h_c GTP C10H12N5O14P3 + H2O H2O --> Dihydromonapterin-triphosphate + Formate + H+
GPR,
Lower bound,0.0
Upper bound,1000.0


6-pyruvate-tetrahydropterin synthase\
dihydromonapterin-triphosphate <-> triphosphate + 6-pyruvoyltetrahydropterin

In [5]:
#add metabolites
six_pyruvoyltetrahydropterin = Metabolite(id='pthp_c', compartment='c')

In [6]:
#design reaction
ptsp = Reaction('ptsp')
ptsp.add_metabolites({model.metabolites.dhmptp_c: -1,
                      model.metabolites.pppi_c: 1,
                      six_pyruvoyltetrahydropterin: 1,
                     })
print(ptsp.build_reaction_string())

dhmptp_c --> pppi_c + pthp_c


In [7]:
#add reaction to the system
model.add_reactions([ptsp])
model.reactions.ptsp

Reaction identifier,ptsp
Name,
Memory address,0x07f2eeff75670
Stoichiometry,dhmptp_c --> pppi_c + pthp_c Dihydromonapterin-triphosphate --> Inorganic triphosphate +
GPR,
Lower bound,0.0
Upper bound,1000.0


sepiapterin reductase\
2 H+ + 2 NADPH + 6-pyruvoyltetrahydropterin <-> 2 NADP+ + tetrahydrobiopterin (BH4)

In [8]:
# add metabolites
tetrahydrobiopterin = Metabolite(id='thbpt_c', compartment='c')

In [9]:
#design reaction
spr = Reaction('spr')
spr.add_metabolites({model.metabolites.h_c: -2,
                      model.metabolites.nadph_c: -2,
                      six_pyruvoyltetrahydropterin: -1,
                      model.metabolites.nadp_c: 2,
                      tetrahydrobiopterin: 1
                     })
print(spr.build_reaction_string())

2 h_c + 2 nadph_c + pthp_c --> 2 nadp_c + thbpt_c


In [10]:
#add reaction to the system
model.add_reactions([spr])
model.reactions.spr

Reaction identifier,spr
Name,
Memory address,0x07f2eeff65a60
Stoichiometry,2 h_c + 2 nadph_c + pthp_c --> 2 nadp_c + thbpt_c 2 H+ + 2 Nicotinamide adenine dinucleotide phosphate - reduced + --> 2 Nicotinamide adenine dinucleotide phosphate +
GPR,
Lower bound,0.0
Upper bound,1000.0


**BH4 regeneration pathway**

pterin-4alpha-carbinolamine dehydratase (PCD)\
4a-hydroxytetrahydrobiopterin (hydro-BH4) <-> dihydrobiopterin (BH2) + water 

In [11]:
#add metabolites
dihydrobiopterin = Metabolite(id='dhbpt_c', compartment='c')
four_alpha_hydroxytetrahydrobiopterin = Metabolite(id='hthbpt_c', compartment='c')

In [12]:
#design reaction
pcd = Reaction('pcd')
pcd.add_metabolites({four_alpha_hydroxytetrahydrobiopterin: -1,
                      dihydrobiopterin: 1,
                      model.metabolites.h2o_c: 1
                     })
print(pcd.build_reaction_string())

hthbpt_c --> dhbpt_c + h2o_c


In [13]:
#add reaction to the system
model.add_reactions([pcd])
model.reactions.pcd

Reaction identifier,pcd
Name,
Memory address,0x07f2eeff65460
Stoichiometry,hthbpt_c --> dhbpt_c + h2o_c --> + H2O H2O
GPR,
Lower bound,0.0
Upper bound,1000.0


dihydropteridine reductase (DHPR)\
dihydrobiopterin (BH2) + H+ + NADPH <-> NADP+ + tetrahydrobiopterin (BH4)

In [14]:
#design reaction
dhpr = Reaction('dhpr')
dhpr.add_metabolites({model.metabolites.dhbpt_c: -1,
                      model.metabolites.h_c: -1,
                      model.metabolites.nadph_c: -1,
                      model.metabolites.nadp_c: 1,
                      model.metabolites.thbpt_c: 1
                     })
print(dhpr.build_reaction_string())

dhbpt_c + h_c + nadph_c --> nadp_c + thbpt_c


In [15]:
#add reaction to the system
model.add_reactions([dhpr])
model.reactions.dhpr

Reaction identifier,dhpr
Name,
Memory address,0x07f2eeff750a0
Stoichiometry,dhbpt_c + h_c + nadph_c --> nadp_c + thbpt_c + H+ + Nicotinamide adenine dinucleotide phosphate - reduced --> Nicotinamide adenine dinucleotide phosphate +
GPR,
Lower bound,0.0
Upper bound,1000.0


**5-HTP synthesis pathway**

tryptophan hydroxylase generates 5-hydroxytryptophan\
oxygen + tryptophan (trp) + tetrahydrobiopterin (BH4) <-> 5-hydroxytryptophan (5-HTP) +  4a-hydroxytetrahydrobiopterin (hydro-BH4)

In [16]:
#add metabolites to the system
hydroxytryptophan = Metabolite(id='htrp_c', compartment='c')

In [17]:
#design reaction
tph = Reaction('tph')
tph.add_metabolites({tetrahydrobiopterin: -1,
                      model.metabolites.o2_c: -1,
                      model.metabolites.trp__L_c: -1,
                      four_alpha_hydroxytetrahydrobiopterin: 1,
                      hydroxytryptophan: 1
                     })
print(tph.build_reaction_string())

o2_c + thbpt_c + trp__L_c --> hthbpt_c + htrp_c


In [18]:
#add reaction to the system
model.add_reactions([tph])
model.reactions.tph

Reaction identifier,tph
Name,
Memory address,0x07f2eeff65790
Stoichiometry,o2_c + thbpt_c + trp__L_c --> hthbpt_c + htrp_c O2 O2 + + L-Tryptophan --> +
GPR,
Lower bound,0.0
Upper bound,1000.0


**Remove 5-HTP from the system**

In [19]:
five_hydroxytryptophan_exchange = Reaction('EX_htrp')
five_hydroxytryptophan_exchange.add_metabolites({model.metabolites.htrp_c: -1})
model.add_reaction(five_hydroxytryptophan_exchange)

**Optimize model**

In [20]:
print(model.objective)

Maximize
1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685


In [23]:
# writing the model
write_sbml_model(model, "../data/iML1515_5htp_2nd_strategy.xml")